In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser 
from langchain_community.document_loaders.csv_loader import CSVLoader
import os  # 환경 변수 설정을 위한 모듈

# OpenAI API 키 설정 (실제 사용 시에는 안전한 방법으로 API 키를 관리해야 함)
os.environ["OPENAI_API_KEY"] = ""

In [2]:
# 1. CSV 파일 로드
# - CSVLoader를 사용하여 지정된 경로의 CSV 파일을 로드
# - 각 행(row)이 하나의 문서(Document)로 변환됨
loader = CSVLoader(file_path="../data/titanic.csv")
docs = loader.load()
print(f"Row 수: {len(docs)}")

Row 수: 891


In [7]:
# 2. 불러온 문서 내용 확인
# - 특정 문서의 내용을 출력하여 데이터 확인
print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")


[페이지내용]
PassengerId: 11
Survived: 1
Pclass: 3
Name: Sandstrom, Miss. Marguerite Rut
Sex: female
Age: 4
SibSp: 1
Parch: 1
Ticket: PP 9549
Fare: 16.7
Cabin: G6
Embarked: S

[metadata]
{'source': '../data/titanic.csv', 'row': 10}



In [8]:
# 3. 문서를 작은 청크로 분할
# - RecursiveCharacterTextSplitter를 사용하여 텍스트를 일정 크기의 청크로 나눔
# - chunk_size: 한 개의 청크 크기 (문자 수)
# - chunk_overlap: 청크 간 겹치는 부분의 크기 (연속성 유지)
chunk_size = 1000  # 적절한 값 조절 필요
chunk_overlap = 100  # 적절한 값 조절 필요
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

# 문서를 여러 개의 작은 청크로 나눔 
splits = text_splitter.split_documents(docs)
print(f"청크 개수: {len(splits)}")  # 생성된 청크 개수 확인

청크 개수: 891


In [18]:
# 4. 벡터스토어(Vector Store) 생성
# - FAISS(Facebook AI Similarity Search)를 사용하여 벡터 DB를 생성
# - OpenAIEmbeddings를 사용하여 텍스트를 벡터로 변환하여 저장
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))

In [19]:
# 5. 문서에서 정보를 검색할 수 있도록 검색기(retriever) 생성
retriever = vectorstore.as_retriever()

In [20]:
# 6. 프롬프트 템플릿 정의 (출처 포함)
# - 검색된 문서를 기반으로 질문에 답변하도록 설계
prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context)에서 주어진 질문(question)에 답하는 것입니다.
검색된 다음 문맥(context)을 사용하여 질문(question)에 답하세요. 만약, 주어진 문맥(context)에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다`라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.
#Question:
{question}

#Context:
{context}

#Answer:"""
)

In [21]:
# 7. LLM 모델 선택
# - OpenAI의 GPT-4o-mini 모델을 사용하여 검색된 정보를 바탕으로 응답을 생성
# - temperature=0 설정으로 일관된 답변을 생성하도록 설정
llm = ChatOpenAI(model_name="gpt-5-nano", temperature=1)

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_25644\2569727233.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-5-nano", temperature=1)


In [22]:
# 8. RAG(Retrieval-Augmented Generation) 체인 생성
# - 검색된 문서를 바탕으로 LLM이 답변을 생성하도록 연결
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}  # 검색을 수행하는 객체 입력
    | prompt
    | llm
    | StrOutputParser()
)

In [23]:
# 실행 예시 (사용자가 입력한 질문)
user_question = "데이터에 대해서 알려주세요."
response = rag_chain.invoke(user_question)  # 사용자의 질문을 입력
print(response)  # 최종 생성된 답변 출력

주어진 컨텍스트는 Titanic 데이터셋(titanic.csv)의 일부 레코드를 보여주는 내용입니다. 주요 특징은 아래와 같습니다.

- 컬럼(열) 구성
  - PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked 등이 포함되어 있습니다.
  - Survived는 생존 여부를 나타내며 0/1 값으로 표시됩니다.
  - Pclass는 객실 등급(1, 2, 3)을 나타냅니다.
  - Embarked는 탑승 항구를 나타냅니다(예: S, C, Q).

- 예시 레코드(샘플 4건)
  - 모두 남성(Male)이고 Pclass는 3, Embarked는 S(Southampton)로 공통점이 있습니다.
  - Survived는 모두 0으로 기록되어 있습니다(생존 실패).
  - 연령(Age)은 각각 19, 21, 32, 34로 다양합니다.
  - SibSp(형제/배우자 동반 승객 수)와 Parch(부모/자식 동반 승객 수) 값은 0 또는 1로 나타납니다.
  - Fare는 6.4958 ~ 15.85 범위로 다양합니다.
  - Cabin 값은 비어 있는 경우가 있습니다.

- 데이터 원천 및 메타데이터
  - 소스: ../data/titanic.csv
  - 각 문서는 서로 다른 row 번호를 가리키며, 예시로 687, 623, 206, 202 등의 row가 포함됩니다.

- 활용 가능성
  - 생존 예측 모델링, 탐색적 데이터 분석(EDA), 특성 엔지니어링(예: 가족 동반 여부, 탑승 항구, Fare 범주화) 등에 사용될 수 있습니다.
  - 주어진 샘플은 전체 데이터의 일부이므로 대표성을 일반화하기에는 한정적입니다.

간단히 말해, 이 데이터는 Titanic 승객의 기본 정보를 담고 있으며, 생존 여부를 예측하는 전형적인 구조의 데이터 세트에서 일부 샘플을 확인할 수 있습니다.


In [25]:
# 실행 예시 (사용자가 입력한 질문)
user_question = "이 데이터를 가지고 어떤 걸 분석할 수 있을지 아이디어 5개 리스트 형식으로 정리해줘."
response = rag_chain.invoke(user_question)  # 사용자의 질문을 입력
print(response)  # 최종 생성된 답변 출력

다음 다섯 가지 아이디어로 분석해 볼 수 있습니다.

1) 생존 예측 요인 파악
- Survived를 종속 변수로 삼고 Pclass, Sex, Embarked, Fare, Age 등 피처의 영향력을 파악하며 간단한 예측 모델(예: 로지스틱 회귀)이나 시각화로 중요한 요인을 식별.

2) Age 결측치 처리 및 연령별 생존률 분석
- Age의 결측치를 어떻게 처리할지 실험하고, Age 그룹(예: 0-12, 13-20, 21-40, 41-60, 60+)에 따른 생존률을 비교.

3) 가족 규모(FamilySize)와 생존의 관계
- FamilySize = SibSp + Parch + 1로 파생 피처를 만들고, 가족 구성원 수에 따른 생존률 차이와 트렌드를 분석.

4) Fare와 Pclass의 상호작용 및 생존 분석
- Pclass별 Fare 분포를 확인하고, Fare 구간에 따른 생존률을 분석하여 사회경제적 위치가 생존에 미친 영향을 탐색.

5) Name에서 Title 추출을 통한 사회경제적 지위 분석
- Name에서 Title(Mr., Mrs., Miss 등)을 추출해 Title별 생존률과 Sex와의 교차 관계를 분석하여 사회적 지위나 역할 차이가 생존에 미친 영향을 살펴봄.
